In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root='../data/',
    train=True,
    download=False,
    transform=ToTensor()
)
test_data = datasets.FashionMNIST(
    root='../data/',
    train=False,
    download=False,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5
loss_fn = nn.CrossEntropyLoss()


In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### 训练循环中，优化器分三个步骤进行：
调用optimizer.zero_grad()来重置模型梯度，防止梯度累加。<br>
调用loss.backward()进行反向传播,将损失关于每个参数的梯度保存下来。<br>
反向传播得到新的梯度后，调用optimizer.step()根据反向传播收集的梯度调整参数。

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

In [8]:
def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy:{(100*correct):>0.1f}%, Avg loss:{test_loss:>8f} \n')


In [9]:
for t in range(10):
    print(f"Epoch {t + 1}\n---------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
---------------------------
loss:2.319683 [   64/60000]
loss:2.288247 [ 6464/60000]
loss:2.283911 [12864/60000]
loss:2.261475 [19264/60000]
loss:2.241156 [25664/60000]
loss:2.234566 [32064/60000]
loss:2.208355 [38464/60000]
loss:2.195540 [44864/60000]
loss:2.157578 [51264/60000]
loss:2.160334 [57664/60000]
Test Error: 
 Accuracy:46.9%, Avg loss:2.147667 

Epoch 2
---------------------------
loss:2.133830 [   64/60000]
loss:2.104510 [ 6464/60000]
loss:2.119761 [12864/60000]
loss:2.077475 [19264/60000]
loss:2.009639 [25664/60000]
loss:2.021829 [32064/60000]
loss:1.979464 [38464/60000]
loss:1.910437 [44864/60000]
loss:1.913360 [51264/60000]
loss:1.913972 [57664/60000]
Test Error: 
 Accuracy:54.0%, Avg loss:1.853683 

Epoch 3
---------------------------
loss:1.890594 [   64/60000]
loss:1.761734 [ 6464/60000]
loss:1.830572 [12864/60000]
loss:1.712128 [19264/60000]
loss:1.684302 [25664/60000]
loss:1.662414 [32064/60000]
loss:1.625896 [38464/60000]
loss:1.538617 [44864/60000]
loss:1.4

In [10]:
import torchvision.models as models

In [27]:
model_path = '../outputs/checkpoints/model_weights.pth'
model = models.vgg16(weights='IMAGENET1K_V1')
print(model)
torch.save(model.state_dict(), '../outputs/checkpoints/model_weights.pth')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [28]:
path = '../outputs/checkpoints/model_weights.pth'
loaded_model = models.vgg16()
loaded_model.load_state_dict(torch.load(path))
loaded_model._load_state_dict_post_hooks.clear()
print(loaded_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1